In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.set_option('display.max_rows', 104)
pd.set_option('display.max_columns',104)
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import RFE, RFECV
from sklearn.metrics import f1_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from sklearn.feature_selection import RFE, RFECV
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_score, recall_score
import warnings
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)

In [ ]:
df['Credit_Product']  =df['Credit_Product'].fillna('Maybe')
df.isna().sum()

In [ ]:
df['Is_Lead'].value_counts() # 1:3 ratio moderate imbalance

In [ ]:
numerical_features = df.select_dtypes(include=np.number)
categorical_features = df.select_dtypes(include=np.object)
print("numeric_features: ", numerical_features.shape, numerical_features.columns)
print("categorical_features: ", categorical_features.shape, categorical_features.columns)

# Numercial Data Anlysis

In [ ]:
numerical_features.describe()

In [ ]:

# skewness & kurtosis
pd.DataFrame({"Skewness": df.skew(), "Kurtosis": df.kurt()})

If skewness is less than −1 or greater than +1, the distribution is highly skewed. If skewness is between −1 and −½ or between +½ and +1, the distribution is moderately skewed. If skewness is between −½ and +½, the distribution of Avg account balance is highly skewed hence we need to do rectify this

In [ ]:
fig, axs = plt.subplots(2,2, figsize=(16, 6), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()

for i,j in zip(numerical_features.columns.to_list(),range(4)):
    axs[j].hist(numerical_features[i])
    axs[j].set_title(i+': '+str(np.round(numerical_features[i].skew(),2)))

In [ ]:
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(14,20))
AX = [ax1, ax2, ax3, ax4]

for i,j in zip(numerical_features.columns.to_list(),AX):
    sns.boxplot(x = 'Is_Lead', y = i, data =numerical_features,ax=j)


If age and vintage is high. credit card lead is high

In [ ]:
# Correlation matrix
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(numerical_features.corr())

ageand vintage is slightly correlated

In [ ]:
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(numerical_features.values, i) for i in range(numerical_features.shape[1])]
vif["features"] = numerical_features.columns
vif

# Categorical Analysis

In [ ]:
for i in categorical_features.columns.to_list():
    print("Total unique values for",i,len(categorical_features[i].unique()))
    print("Value Counts for",i,'\n',categorical_features[i].value_counts(),'\n')

In [ ]:
fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(nrows=3, ncols=2, figsize=(14,10))
AX = [ax1, ax2, ax3, ax4, ax5, ax6]

for i,j in zip(categorical_features.columns.to_list(),AX):
    temp = pd.crosstab(df[i],df['Is_Lead'])
    temp.plot(kind='bar',stacked=True,color=['red','green'],grid=False,ax=j)

If we dont have credit product info then there is higher chance of crdit card lead and some location have higher chances of lead

In [ ]:
for i in categorical_features.columns.to_list():
    print(pd.crosstab(df[i],df['Is_Lead']))

# Feature Engineering 

In [ ]:
df = pd.read_csv('/kaggle/input/analytics-vidhya-jobathon-may2021/job_may_train.csv',index_col=0)
test = pd.read_csv('/kaggle/input/analytics-vidhya-jobathon-may2021/job_may_test.csv',index_col=0)
df['source'] = 'train'
test['source'] = 'test'
data = pd.concat([df,test],ignore_index=True)
data['Credit_Product'].replace(np.nan,'Maybe',inplace=True)

In [ ]:
sns.distplot(data['Avg_Account_Balance'])
plt.show()

Data is +vely skewed hence doing log multiplication

In [ ]:
data['Avg_Account_Balance'] = np.log(data['Avg_Account_Balance'])
sns.distplot(data['Avg_Account_Balance'])
plt.show()

In [ ]:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

#Drop unnecessary columns:
test.drop(['Is_Lead','source'],axis=1,inplace=True)
train.drop('source',axis=1,inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
var_mod = ['Gender','Region_Code','Occupation','Channel_Code','Credit_Product','Is_Active']
for i in var_mod:
    train[i] = le.fit_transform(train[i])
    test[i] = le.fit_transform(test[i])

# Model Selection

In [ ]:
X = train.drop('Is_Lead',axis=1)
y = train['Is_Lead']

In [ ]:
def simple_model(alg, X,y):

    # splitting data into training and validation set
    xtrain, xtest, ytrain, ytest = train_test_split(X,y, random_state=42, test_size=0.2)
    model = alg
    model.fit(xtrain, ytrain) # training the model

    print("roc_score:",roc_auc_score(ytest, model.predict(xtest))) # calculating f1 score
    print("Accuracy on train data:",model.score(xtrain,ytrain))
    print("Accuracy on test data:",model.score(xtest,ytest))

In [ ]:
algs = [LogisticRegression(),DecisionTreeClassifier(),RandomForestClassifier(),AdaBoostClassifier(), XGBClassifier(),lgb.LGBMClassifier()]
algs_lst = ['LR','DTC','RFC','ABC', 'XGB', 'LGB']
for alg,l in zip(algs,algs_lst):
    print(l)
    simple_model(alg, X, y)

Choosing LGB since getting similar roc score and higher test accuracy and it is faster then xgboost

In [ ]:
# using a strtified fold function


def Strat_Kcross_validation(X, y, model, params, folds=5):
    stkf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=21)
    for fold, (train_index, test_index) in enumerate(stkf.split(X, y)):
        print(f"Fold: {fold}")
        x_train, y_train = X.iloc[train_index], y.iloc[train_index]
        x_test, y_test = X.iloc[test_index], y.iloc[test_index]

        alg = model(**params)
        alg.fit(x_train, y_train,
                eval_set=[(x_test, y_test)],
                early_stopping_rounds=100,
                verbose=400)

        pred = alg.predict_proba(x_test)[:, 1]
        roc_score = roc_auc_score(y_test, pred)
        print(f"roc_auc_score: {roc_score}")
        print("-"*50)
    
    return alg


In [ ]:
lgb_params= {'learning_rate': 0.07, 
             'n_estimators': 20000, 
             'max_bin': 86,
             'num_leaves': 10, 
             'max_depth': 27, 
             'reg_alpha': 8.5, 
             'reg_lambda': 6.4, 
             
             'scale_pos_weight':1.3
            }

lgb_model = Strat_Kcross_validation(X, y, LGBMClassifier, lgb_params)

In [ ]:
pred_test_lgb = lgb_model.predict_proba(test)[:,1]
sample_submission = pd.read_csv(r'submission.csv')
sample_submission['Is_Lead'] = pred_test_lgb
sample_submission.to_csv(f'pred_test_final.csv',index=False)

In [ ]:
lgb_params= {'learning_rate': 0.07, 
             'n_estimators': 50000, 
             'max_bin': 86,
             'num_leaves': 10, 
             'max_depth': 27, 
             'reg_alpha': 8.5, 
             'reg_lambda': 6.4, 
             
             'scale_pos_weight':1.35
            }

lgb_model = Strat_Kcross_validation(X, y, LGBMClassifier, lgb_params)

In [ ]:
pred_test_lgb = lgb_model.predict_proba(test)[:,1]
sample_submission = pd.read_csv(r'/kaggle/input/analytics-vidhya-jobathon-may2021/job_may_submission.csv')
sample_submission['Is_Lead'] = pred_test_lgb
sample_submission.to_csv(f'pred_test_fin.csv',index=False)